<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [595]:
import pandas as pd
import psycopg2

In [596]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432 

In [597]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [598]:
       
query_3_1 = f'''select 
                     count(v.id)
            from public.vacancies v 
            '''
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [600]:
# текст запроса
query_3_2 = f'''select 
                     count(e.id)
            from public.EMPLOYERS e 
            '''
df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [602]:
query_3_3 = f'''select 
                     count(a.id)
            from public.areas a 
            '''
df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [604]:
# текст запроса
query_3_4 = f'''select 
                     count(i.id)
            from public.INDUSTRIES i 
            '''
df = pd.read_sql_query(query_3_4, connection)
df

,count
0,294


***

1. Количество вакансий в нашей базе 49197, количество работадателей 23501, 
следовательно у одного работадателя может быть несколько вакансий.
2. Количество регионов 1362.
3. Количество сфер деятельности 294.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [702]:
querty_3_5 = f'''select a.name area,
                        count(v.id) cnt                        
                from public.vacancies v
                join public.AREAS a on v.area_id = a.id 
                group by a.name
                order by 2 desc'''
                
df = pd.read_sql_query(querty_3_5, connection)
df


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [608]:
import warnings 
warnings.filterwarnings('ignore')

In [703]:
querty_3_5 = f'''select v.salary_from,
                        v.salary_to,
                        count(v.id) cnt                        
                from public.vacancies v
                group by v.salary_from, v.salary_to
                having v.salary_from is not null or v.salary_to is not null
                order by 3 desc'''
                
df = pd.read_sql_query(querty_3_5, connection)
df['cnt'].sum()

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [704]:
querty_3_5 = f'''select avg(v.salary_from) mean_salary_from,
                        avg(v.salary_to) mean_salary_to                        
                from public.vacancies v
                '''
df = pd.read_sql_query(querty_3_5, connection)
print('среднее значение нижней границы зп:', int(df.loc[0, 'mean_salary_from'].round()))
print('среднее значение верхней границы зп:', int(df.loc[0, 'mean_salary_to'].round()))

среднее значение нижней границы зп: 71065
среднее значение верхней границы зп: 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [705]:
querty_3_5 = f'''select v.schedule,
                        v.employment,
                        count(v.id) cnt                        
                from public.vacancies v
                group by v.schedule, v.employment
                order by 3 desc
                --offset 1 limit 1'''
                
df = pd.read_sql_query(querty_3_5, connection)
df

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [706]:
querty_3_5 = f'''select v.experience,
                        count(v.id) cnt                        
                from public.vacancies v
                group by v.experience
                order by 2
                '''
                
df = pd.read_sql_query(querty_3_5, connection)
df

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

1. Больше всего вакансий в столицах и крупных городах (Москва, Санкт-Петербург, Минск, Новосибирск, Алматы и т.д.)
2. У большей половины вакансий не указан ни один из параметром заработной платы
3. Из тех цифр, которые указаны, средняя зарплата специалистов в районе 71000 - 110000 руб.
4. Чаще всего требуется работники на тип трудоустройства "полная занятость", при графике работы по убыванию количества вакансий: "Полный день", "Удаленная работа", "Гибкий график".
5. Наиболе часто встречаемый опыт работы - "От 1 года до 3 лет", наименее - "Более 6 лет".

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [709]:
#для вывода непоследовательно расположенных строк, использовала соединение таблиц
querty_3_5 = f'''(select e.name,
                        count(v.id) cnt                        
                from public.EMPLOYERS e
                JOIN public.vacancies v on v.employer_id = e.id
                group by e.name
                order by 2 desc
                offset 4 limit 1)
                union 
                (select e.name,
                        count(v.id) cnt                        
                from public.EMPLOYERS e
                JOIN public.vacancies v on v.employer_id = e.id
                group by e.name
                order by 2 desc
                limit 1)
                               
                '''
df = pd.read_sql_query(querty_3_5, connection)
df                

,name,cnt
0,Газпром нефть,331
1,Яндекс,1933


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [712]:
querty_3_5 = f'''select distinct a.name as region, 
                        count(e.id) as employers_count, 
                        count(v.id) as vacancies_count
                from public.areas a 
                left join public.employers e on e.area=a.id 
                left join public.vacancies v on v.area_id=a.id
                where v.id is null
                group by a.id
                order by count(e.id) desc
                limit 1
                '''

df = pd.read_sql_query(querty_3_5, connection)
df


,region,employers_count,vacancies_count
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [713]:
querty_3_5 = f'''select distinct e.name as employer_name, 
                        count(distinct v.area_id) as area_count
                from public.employers e 
                left join public.vacancies v on v.employer_id= e.id
                group by e.id
                order by 2 desc
                '''
                
df = pd.read_sql_query(querty_3_5, connection)
df

,employer_name,area_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23336,Ярпож Казань,0
23337,ЯРСНИП,0
23338,Ясли-сад № 28 г.Минска,0
23339,Яшин&Партнёры,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [714]:
querty_3_5 = f'''select 
                        count(e.id) employer_cnt
                from public.employers e 
                left join public.EMPLOYERS_INDUSTRIES i on i.employer_id= e.id
                where i.industry_id is null
                '''
df = pd.read_sql_query(querty_3_5, connection)
df

,employer_cnt
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [715]:
querty_3_5 = f'''select e.name employer_name,
                        count(i.industry_id) industry_cnt
                from public.employers e 
                left join public.EMPLOYERS_INDUSTRIES i on i.employer_id = e.id
                group by e.id
                having count(i.industry_id) = 4
                order by e.name
                offset 2 limit 1
                '''
                
df = pd.read_sql_query(querty_3_5, connection)
df

,employer_name,industry_cnt
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [716]:
querty_3_5 = f'''select 
                        count(e.employer_id) employer_cnt
                from public.EMPLOYERS_INDUSTRIES e 
                join public.INDUSTRIES i on e.industry_id = i.id
                WHERE i.name = 'Разработка программного обеспечения'
                '''
df = pd.read_sql_query(querty_3_5, connection)
df                


,employer_cnt
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [717]:
city = (
    "Москва", "Санкт-Петербург", "Новосибирск", 
    "Екатеринбург", "Казань", "Нижний Новгород", 
    "Красноярск", "Челябинск", "Самара", "Уфа", 
    "Ростов-на-Дону", "Омск", "Краснодар", 
    "Воронеж", "Пермь", "Волгоград"
)

querty = f'''(select a.name city,
                    count(v.id) vacancies_cnt
                from public.EMPLOYERS e 
                join public.VACANCIES v on e.id = v.employer_id
                join public.areas a on a.id = v.area_id
                WHERE a.name in {city} and e.name = 'Яндекс'
                group by a.id
                order by 2)
                
                UNION all
                
                (select 'total',
                    count(v.id) 
                 from public.EMPLOYERS e 
                join public.VACANCIES v on e.id = v.employer_id
                join public.areas a on a.id = v.area_id
                WHERE a.name in {city} and e.name = 'Яндекс')
                '''
                
df = pd.read_sql_query(querty, connection)
df

,city,vacancies_cnt
0,total,485
1,Омск,21
2,Челябинск,22
3,Красноярск,23
4,Волгоград,24
5,Ростов-на-Дону,25
6,Казань,25
7,Пермь,25
8,Самара,26
9,Уфа,26


***

1. Топ-5 работадателей: Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нефть
2. По распространенности вакансий лидируют: Яндекс, Ростелеком, Спецремонт
3. У одного работодателя может быть увазано несколь сфер деятельности или не указано ни одной.
4. Компания Яндекс ищет вакансии во всех крупных городах-миллиониках РФ, лидируют: Москва и Санкт-Петербург

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [718]:
querty = f'''select
                    count(v.id) vacancies_cnt
            from public.VACANCIES v
            where v.name ilike '%data%' or v.name ilike '%данн%'
            '''
            
df = pd.read_sql_query(querty, connection)
df

,vacancies_cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [719]:
querty = f''' 
            select
                count(v.id) df_jun_vacancies
            from public.vacancies v
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and (v.name ilike '%junior%' 
            or v.experience ilike '%нет%опыта%' 
            or v.employment ilike '%стажировка%')
                '''
                
df = pd.read_sql_query(querty, connection)
df

,df_jun_vacancies
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [720]:
querty = f''' 
            select
                count(v.id) df_sql_vacancies
            from public.vacancies v
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and (v.key_skills ilike '%sql%' 
            or v.key_skills ilike '%postgres%')
                '''
                
df = pd.read_sql_query(querty, connection)
df

,df_sql_vacancies
0,201


In [721]:
querty = f''' 
            select
                count(v.id) df_phyton_vacancies
            from public.vacancies v
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and v.key_skills ilike '%python%'
                '''
df = pd.read_sql_query(querty, connection)
df

,df_phyton_vacancies
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [723]:
#Для определения количества навыков вычисляю число разделителей, с помощью замены на пустую строку. 
#Затем прибаляю 1, так как если есть хотя бы 1 разделитель, то навыков на единицу больше.
#Далее привожу к числовому типу для вычисления среднего показателя и округления до сотых.
querty = f''' 
            select 
                   ROUND(AVG((length(key_skills) - length(REPLACE(key_skills, CHR(9), '' ))) + 1)::numeric, 2) cnt_df_key_skills
            from public.vacancies v
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and v.key_skills is not null
            
                '''
df = pd.read_sql_query(querty, connection)
df

,cnt_df_key_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [698]:
querty = f''' 
            select round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))
            from public.vacancies v                   
            where (v.name ilike '%data scientist%'
            or v.name ilike '%data science%' 
            or v.name ilike '%исследователь данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine learning%'
            or v.name ilike '%машинн%обучен%')
            and (v.salary_from is not null 
            or v.salary_to is not null)
            and v.experience ilike '%от 3 до 6 лет%'
            
                '''
                
df = pd.read_sql_query(querty, connection)
df

***

1. Общее число вакансий, имеющее отношение к данным = 1771
2. Число вакансий для начинающий специалистов = 51
3. Знание SQL/Postgres требуется в 201 вакансии, знание Phython = 351
4. Количество среднее требуемых навыков от специалиста = 6,41
5. Средняя заработная плата специалистов в сфере data science c опытом работы 3-6 лет = 243115 руб. 

# Дополнительно

In [646]:
# исследование количества вакансий для начинающих специалистов в сфере исследования данных и ML по различным компаниям
querty = f''' 
            select
                e.name EMPLOYERS_name,
                count(v.id) df_jun_vacancies
            from public.vacancies v
            join public.EMPLOYERS e on e.id = v.employer_id
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and (v.name ilike '%junior%' 
            or v.experience ilike '%нет%опыта%' 
            or v.employment ilike '%стажировка%')
            group by e.id
            order by 2 desc
                '''
df = pd.read_sql_query(querty, connection)
df               
                

,employers_name,df_jun_vacancies
0,Банк ВТБ (ПАО),10
1,Comexp,3
2,The Skolkovo Institute of Science and Technology,3
3,"СтарЛайн, Научно-производственное объединение",2
4,SteadyControl,2
5,Диплей,2
6,VK,1
7,"Миррико, Группа Компаний",1
8,Улыбка радуги,1
9,"МАГНИТ, Розничная сеть",1


In [653]:
#вычесление средней заработной платы для начинающих специалистов в сфере исследования данных и ML
querty = f''' 
            select round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)))
            from public.vacancies v                   
            where (v.name ilike '%data scientist%'
            or v.name ilike '%data science%' 
            or v.name ilike '%исследователь данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine learning%'
            or v.name ilike '%машинн%обучен%')
            and (v.salary_from is not null 
            or v.salary_to is not null)
            and (v.name ilike '%junior%' 
            or v.experience ilike '%нет%опыта%' 
            or v.employment ilike '%стажировка%')
            
                '''
df = pd.read_sql_query(querty, connection)
df                 

,round
0,70278.0


In [657]:
# исследование количества вакансий у различных компаний для начинающих специалистов в сфере исследования данных и ML
querty = f''' 
            select 
                i.name industry_name,
                count(i.name) cnt_industries
            from public.industries i
            join public.employers_industries ei on i.id = ei.industry_id
            join public.vacancies v on ei.employer_id = v.employer_id
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and v.key_skills is not null
            and (v.name ilike '%junior%' 
            or v.experience ilike '%нет%опыта%' 
            or v.employment ilike '%стажировка%')
            group by i.id
            order by 2 desc
            
                '''
df = pd.read_sql_query(querty, connection)
df                   

,industry_name,cnt_industries
0,Разработка программного обеспечения,16
1,Банк,11
2,"Управляющая, инвестиционная компания (управлен...",10
3,"Системная интеграция, автоматизации технологи...",8
4,"Интернет-компания (поисковики, платежные систе...",7
5,"Научно-исследовательская, научная, академическ...",5
6,Консалтинговые услуги,4
7,"Электронно-вычислительная, оптическая, контрол...",3
8,"Автокомпоненты, запчасти (производство)",2
9,"Бытовая техника, электроника, климатическое об...",2


In [667]:
#определение наличия сфер деятельности для специалистов в сфере исследования данных и ML в здравоохранении
querty = f''' 
            select 
                i.name industry_name,
                count(i.name) cnt_industries
            from public.industries i
            join public.employers_industries ei on i.id = ei.industry_id
            join public.vacancies v on ei.employer_id = v.employer_id
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and v.key_skills is not null
            and (i.name ilike '%здрав%' 
            or i.name ilike '%здоров%'
            or i.name ilike '%мед%'
            or i.name ilike '%врач%')
            group by i.id
            order by 2 desc
            
                '''
df = pd.read_sql_query(querty, connection)
df                   

,industry_name,cnt_industries
0,"Производство мультимедиа, контента, редакторск...",8
1,Распространение мультимедиа и печатной продукции,1


In [701]:
#исследования количтсва ваканасий с удаленным типом работы для начинающих специалистов в сфере исследования данных и ML
querty = f'''select distinct a.name as area_name, 
                        count(a.name) as area_count
                from public.areas a 
                join public.vacancies v on v.area_id= a.id
                where (v.name ilike '%data%scientist%'
                or v.name ilike '%data%science%' 
                or v.name ilike '%исследователь%данных%'
                or (v.name like '%ML%' and v.name not ilike '%HTML%') 
                or v.name ilike '%machine%learning%'
                 or v.name ilike '%машинн%обучен%')
                and v.key_skills is not null
                and (v.name ilike '%junior%' 
                or v.experience ilike '%нет%опыта%' 
                or v.employment ilike '%стажировка%')
                and v.schedule ilike '%Удаленная работа%'
                group by a.id
                order by 2 desc
                '''
df = pd.read_sql_query(querty, connection)
df 

,area_name,area_count
0,Москва,2
1,Воронеж,1
2,Екатеринбург,1
3,Казань,1
4,Минск,1


In [680]:
# вычисление среднего числа ключевых навков для начинающих специалистов в сфере исследования данных и ML
querty = f''' 
            select 
                   ROUND(AVG((length(key_skills) - length(REPLACE(key_skills, CHR(9), '' ))) + 1)::numeric, 2) cnt_df_key_skills
            from public.vacancies v
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and v.key_skills is not null
            and (v.name ilike '%junior%' 
            or v.experience ilike '%нет%опыта%' 
            or v.employment ilike '%стажировка%')
            
            
            
                '''
df = pd.read_sql_query(querty, connection)
df                   

,cnt_df_key_skills
0,7.27


In [684]:
# а также, для сравнения, вычисление среднего числа ключевых навков для специалистов в сфере исследования данных и ML с другим опытом работы
querty = f''' 
            select 
                   ROUND(AVG((length(key_skills) - length(REPLACE(key_skills, CHR(9), '' ))) + 1)::numeric, 2) cnt_df_key_skills
            from public.vacancies v
            where (v.name ilike '%data%scientist%'
            or v.name ilike '%data%science%' 
            or v.name ilike '%исследователь%данных%'
            or (v.name like '%ML%' and v.name not ilike '%HTML%') 
            or v.name ilike '%machine%learning%'
            or v.name ilike '%машинн%обучен%')
            and v.key_skills is not null
            and v.experience ilike '%от 3 до 6 лет%'
            
            
                '''
df = pd.read_sql_query(querty, connection)
df 

,cnt_df_key_skills
0,5.99


In [677]:
 #исследование распределения количества различных вакансий в сфере IT
querty = f''' 
            select 
                   v.name vacancies_name,
                   count(v.name)
            from public.vacancies v
            group by v.name
            order by 2 desc
            limit 20
                '''
df = pd.read_sql_query(querty, connection)
df                   

,vacancies_name,count
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


In [688]:
#исследований количество вакансий для специалистов DS в различных компаниях
querty = f'''select distinct e.name as employer_name, 
                        count(v.id) as cnt_vacancies
                from public.employers e 
                join public.vacancies v on v.employer_id= e.id
                where (v.name ilike '%data%scientist%'
                or v.name ilike '%data%science%' 
                or v.name ilike '%исследователь%данных%'
                or (v.name like '%ML%' and v.name not ilike '%HTML%') 
                or v.name ilike '%machine%learning%'
                or v.name ilike '%машинн%обучен%')
                group by e.id
                order by 2 desc
                limit 15
                '''
                
df = pd.read_sql_query(querty, connection)
df 

,employer_name,cnt_vacancies
0,СБЕР,38
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,EvenBet Gaming,9
6,Яндекс,9
7,МегаФон,8
8,Andersen,7
9,Ozon,6


In [695]:
# исследование количества вакансий специалистов DS в разных городах в зависимости графика работы. 
querty = f'''select distinct a.name as area_name, 
                        v.schedule schedule,
                        count(a.name) as area_count
                from public.areas a 
                join public.vacancies v on v.area_id= a.id
                where (v.name ilike '%data%scientist%'
                or v.name ilike '%data%science%' 
                or v.name ilike '%исследователь%данных%'
                or (v.name like '%ML%' and v.name not ilike '%HTML%') 
                or v.name ilike '%machine%learning%'
                or v.name ilike '%машинн%обучен%')
                group by a.id, v.schedule
                order by 3 desc
                limit 15
                '''
df = pd.read_sql_query(querty, connection)
df 

,area_name,schedule,area_count
0,Москва,Полный день,162
1,Москва,Удаленная работа,46
2,Санкт-Петербург,Полный день,45
3,Нижний Новгород,Полный день,14
4,Новосибирск,Полный день,13
5,Алматы,Полный день,12
6,Санкт-Петербург,Удаленная работа,12
7,Москва,Гибкий график,11
8,Казань,Полный день,10
9,Санкт-Петербург,Гибкий график,8


# Общий вывод по проекту

1. Среди всех вакансий для IT сферы, наиболее востребованны системный администратор, программист 1С, бизнес-аналитик, специалист технической поддержки, системный аналитик и аналитик и др.
2. Процент вакансий связанных с data science, исследованием данных  и ML = 3,5% 
3. Топ - 5 компании, в которых требуются данные специалисты: СБЕР, Bell Integrator, Банк ВТБ (ПАО), VK, Positive Technologies.
4. Большее число специалистов DS требуется в Москве на полный рабочий день, на втором месте: Москва - удаленная работа. Далее по убыванию: Санкт-Петербург, Нижний Новгород, Новосибирск, Алматы на полный день.
5. Среднее число навыков, требуемое от начинающего специалиста = 7.27, с опытом работы до 3х лет - 6.73, с опытом  от 3 до 6 лет - 5.99. 
Возможно это связано с тем, что от опытных специалистов требуется более глубокое владение узким набором навыков, а от начинающего специалиста ждут общее понимание принципов работы и базовые знания основных инструментов.
6. Среднияя зарплата начинающих специалистов в сфере data science, исследованием данных  и ML = 70278.0 руб, с опытом до 3х лет = 139675 руб, с опытом от 3 до 6 лет = 243115 руб.

Далее выводы по анализу вакансий для начинающий специалистов без опыта работы, связанных с data science, исследованием данных  и ML:
1. ТОП 7 компании, готовые взять на работу начинающий специалистов DS по убыванию количества вакансий: Банк ВТБ (ПАО), Comexp, The Skolkovo Institute of Science and Technology, СтарЛайн, Научно-производственное объединение, SteadyControl, Диплей, VK.
2. ТОП 7 сфер, в которые требуются начинающие специалисты DS: Разработка программного обеспечения, Банк, Управляющая, инвестиционная компания (управлен..., Системная интеграция, автоматизации технологи..., Интернет-компания (поисковики, платежные систе..., Научно-исследовательская, научная, академическ..., Консалтинговые услуги.
3. Единичные вакансии встречаются с графиком 'Удаленная работа' в 5 городах.

Дополнительно мне интересно было узнать, есть ли среди вакансий сферы деятельности, связанные с медициной. По запросу таких сфер не найдено.

 

